This blog post describes how to use the bokeh plotting library to create some interactive plots

### Run boilerplate code

In [28]:
%run genomeplot.ipynb

In [2]:
# output live to notebook
output_notebook()

Loading BokehJS ...

### Load ag1k data

In [3]:
sys.path.insert(0, '../../../selection_paper/agam-report-base/src/python')
ag1k_dir = '/kwiat/vector/ag1000g/release'
from ag1k import phase1_ar3

In [4]:
phase1_ar3.init(os.path.join(ag1k_dir, 'phase1.AR3'))

In [5]:
chromosomes = "2R", "2L", "3R", "3L", "X"

### Define genome plot function

In [6]:
# Custom changes to axes that pertain to genome not data
# eg in this case move left chromosome arm axes to RHS
def custom_plot(chrom, subplot):
    if chrom.endswith("L"):
        subplot.yaxis.visible = False
        subplot.add_layout(LinearAxis(), 'right')

### Calculate $\theta$, $\Pi$, and Tajima's D

In [7]:
@functools.lru_cache()
def calculate_summary_stats(chrom, pop, window_size=100000):
    
    ix = phase1_ar3.df_samples.query("population == @pop").index
    accessibility = phase1_ar3.accessibility[chrom]["is_accessible"][:]
    
    pos = allel.SortedIndex(phase1_ar3.callset_pass[chrom]["variants/POS"][:])
    eqw = allel.equally_accessible_windows(accessibility, window_size)
    g = allel.GenotypeChunkedArray(phase1_ar3.callset_pass[chrom]["calldata/genotype"]).take(ix, axis=1)
    ac = g.count_alleles()
    
    theta, wins, nb, counts = allel.stats.windowed_watterson_theta(pos, ac, windows=eqw, is_accessible=accessibility)
    
    pi, wins, nb, counts = allel.stats.windowed_diversity(pos, ac, windows=eqw, is_accessible=accessibility)
    
    tajD, wins, counts = allel.stats.windowed_tajima_d(pos, ac, windows=eqw)
    
    df = pd.DataFrame.from_dict({"start": eqw[:, 0], 
                                 "stop": eqw[:, 1], 
                                 "diversity": pi, 
                                 "tajimaD": tajD, 
                                 "theta": theta})

    df["midpoint"] = eqw.mean(1)
    
    return df

In [8]:
stats = {c: calculate_summary_stats(chrom=c, pop="BFS", window_size=100000) for c in chromosomes}

### Use a .gff3 file to annotate above windows

In [9]:
gff3 = allel.FeatureTable.from_gff3(phase1_ar3.geneset_agamp42_fn, attributes=["ID"])

In [10]:
gff3

<FeatureTable shape=(175804,) dtype=(numpy.record, [('seqid', 'O'), ('source', 'O'), ('type', 'O'), ('start', '<i8'), ('end', '<i8'), ('score', '<f8'), ('strand', 'O'), ('phase', '<i8'), ('ID', 'O')])>
[('2L', 'VectorBase', 'contig',        1, 49364325, -1., '.', -1, '2L')
 ('2L', 'VectorBase', 'exon',   157348,   157623, -1., '-', -1, 'AGAP004677-RB-E4A')
 ('2L', 'VectorBase', 'exon',   157348,   157623, -1., '-', -1, 'AGAP004677-RB-E4B')
 ...,
 ('X', 'VectorBase', 'gene', 24338771, 24340371, -1., '+', -1, 'AGAP013609')
 ('X', 'VectorBase', 'rRNA', 24338771, 24340371, -1., '+', -1, 'AGAP013609-RA')
 ('Y_unplaced', 'VectorBase', 'contig',        1,   237045, -1., '.', -1, 'Y_unplaced')]

In [11]:
annotated_data = {}

# annotate these data
for chrom in chromosomes:

    d = stats[chrom].copy()
    
    # extract the relevant seq id and use pandas interval indexing
    features = pd.DataFrame(gff3.query("seqid == '{0}'".format(chrom)).values)
    features.index = pd.IntervalIndex.from_arrays(features.start, features.end, closed="both")

    # logic to extract relevant rows, filter by annot type, drop duplicates and join ID column
    # it would be slightly more efficient to do these both in a single call, but it's fast/readable so we'll let it slide
    d["gene"] = d.apply(
        lambda y: ", ".join(features.loc[[y.start, y.stop]].query("type == 'gene'").ID.drop_duplicates()), 1)

    annotated_data[chrom] = d

In [12]:
annotated_data["X"].head()

,diversity,start,stop,tajimaD,theta,midpoint,gene
0,0.001781,25,132324,-2.190824,0.005357,66174.5,AGAP000011
1,0.003892,132325,246994,-2.041654,0.010288,189659.5,"AGAP000011, AGAP000018"
2,0.005919,246995,368599,-2.176849,0.017548,307797.0,AGAP000018
3,0.006699,368600,487739,-2.297078,0.022272,428169.5,
4,0.009483,487740,625447,-2.252033,0.030146,556593.5,


In [13]:
def stat_plot(chrom, subplot, stat="diversity", label=None):
    
    source = ColumnDataSource(annotated_data[chrom])
    
    if label is None:
        label = stat
    
    hover = HoverTool(
        tooltips=[
            ("Position", "@start{0a.000}-@stop{0a.000}"),
            (label, "$y"),
            ("contig", chrom), 
            ("gene(s)", "@gene")],
        mode="mouse")
    
    subplot.add_tools(hover)
    
    subplot.circle("midpoint", 
                   stat,
                   source=source,
                   size=3, 
                   color="navy", 
                   alpha=0.5)

    subplot.yaxis[0].axis_label = label
    subplot.xaxis[0].axis_label = "Genomic Position (bp)"

In [14]:
gf = GenomePlot(fasta=phase1_ar3.genome_fn, contigs=("2R", "2L", "3R", "3L", "X"), layout="oo|ooo", pfunc=custom_plot)

In [15]:
gf.apply(stat_plot, stat="tajimaD", label="Tajima'sD")

In [16]:
gf.apply(stat_plot, stat="theta", label="\u0398W")

In [120]:
def stat_plot_line(chrom, subplot, stat="diversity", label=None):
    
    source = ColumnDataSource(annotated_data[chrom])
    
    if label is None:
        label = stat
    
    hover = HoverTool(
        tooltips=[
            ("Position", "@start{0a.000}-@stop{0a.000}"),
            (label, "$y"),
            ("contig", chrom), 
            ("gene(s)", "@gene")],
        mode="vline")
    
    subplot.add_tools(hover)
    
    subplot.line("midpoint",
                 stat,
                 source=source,
                 color="navy",
                 alpha=0.5)

    subplot.yaxis[0].axis_label = label
    subplot.xaxis[0].axis_label = "position (bp)"

In [121]:
gf.apply(stat_plot_line, stat="diversity", label="\u03A0")

## Plot G/C content of plasmodium

In [109]:
plasmo_fn = "../assets/PlasmoDB-35_Pfalciparum3D7_Genome.fasta"

In [110]:
fa = pyfasta.Fasta(plasmo_fn, key_fn=lambda y: y.split(" | ")[0])

In [111]:
fa.keys()

KeysView(<pyfasta.fasta.Fasta object at 0x7f63776718d0>)

In [112]:
window_size = 10000

In [113]:
def calc_gc(genome, window_size=10000):
    
    clen = len(genome)
    starts = np.arange(0, clen, window_size)
    stops = starts + window_size
    
    propgc = np.zeros(starts.shape[0])
    
    for i, ix in enumerate(starts):
        r = genome[ix:(ix + window_size)]
        propgc[i] = (r.count("C") + r.count("G")) / (len(r) - r.count("N"))
        
    return pd.DataFrame.from_dict({"start": starts, "stop": stops, "propgc": propgc})

In [114]:
gc_data = {c: calc_gc(fa[c]) for c in fa.keys()}

In [115]:
def plot_gc_data(chrom, subplot):
    
    source = ColumnDataSource(gc_data[chrom])
    
#     if label is None:
#         label = stat
    
    hover = HoverTool(
        tooltips=[
            ("Position", "@start{0a.000}-@stop{0a.000}"),
            ("Pr GC", "$y"),
            ("contig", chrom)],
        mode="vline")
    
    subplot.add_tools(hover)
    
    subplot.line("start",
                 "propgc",
                 source=source,
                 color="navy", 
                 alpha=0.5)

    #subplot.yaxis[0].axis_label = label
    subplot.xaxis[0].axis_label = "position (bp)"

In [116]:
clen = {c: len(fa[c]) for c in fa.keys()}
clen

{'Pf3D7_01_v3': 640851,
 'Pf3D7_02_v3': 947102,
 'Pf3D7_03_v3': 1067971,
 'Pf3D7_04_v3': 1200490,
 'Pf3D7_05_v3': 1343557,
 'Pf3D7_06_v3': 1418242,
 'Pf3D7_07_v3': 1445207,
 'Pf3D7_08_v3': 1472805,
 'Pf3D7_09_v3': 1541735,
 'Pf3D7_10_v3': 1687656,
 'Pf3D7_11_v3': 2038340,
 'Pf3D7_12_v3': 2271494,
 'Pf3D7_13_v3': 2925236,
 'Pf3D7_14_v3': 3291936,
 'Pf3D7_API_v3': 34250,
 'Pf_M76611': 5967}

In [117]:
plas_contigs = sorted(fa.keys())[:14]

In [118]:
gf2 = GenomePlot(plasmo_fn, contigs=plas_contigs, layout="oooooo|ooooo|ooo")
gf2.plot_width_per_mb = 100

In [119]:
gf2.apply(plot_gc_data)

## To do:

- by default only give y-axis to leftmost column
- by default only give x-axis to bottom row
- some logic to determine appropriate X-tick size
- some logic to flag very small contigs, perhaps 10% of size of largest?